## Introduction
I am well aware, that none of us will ever forget coronavirus, but in case someone from another planet reads this(I'm hoping) ,here's an introduction..

Covid-19 is an ongoing global pandemic,caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The virus was first identified in December 2019 in Wuhan, China. The World Health Organization declared a Public Health Emergency of International Concern regarding COVID-19 on 30 January 2020, and later declared a pandemic on 11 March 2020. As of 21 April 2021, more than 143 million cases have been confirmed, with more than 3.04 million deaths attributed to COVID-19, making it one of the deadliest pandemics in history.

![](https://images.unsplash.com/photo-1585913430341-6cd755eae696?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=750&q=80)
*Source:https://unsplash.com/@glencarrie*


In [ ]:
import numpy as np 
import pandas as pd 
from datetime import datetime
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

vaccine = pd.read_csv('/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv')
vaccine['date_new'] = pd.to_datetime(vaccine['date'])
covid=pd.read_csv('../input/covid19-global-dataset/worldometer_coronavirus_daily_data.csv')
covid_cum=pd.read_csv('../input/covid19-global-dataset/worldometer_coronavirus_summary_data.csv')
covid['date_new'] = pd.to_datetime(covid['date'])

**Note**:The main aim of this kernel is not to extensively study the havocs caused by Covid in different countries through the last one year, that is already been done by so many great kernels out there. 



**I'll focus on analyzing the second wave in some countries along with the extent of vaccination.**

## Table of contents
1. [A brief summary of Covid around the world](#summary)"
2. [Second wave in India](#india)
3. [Second Wave in Brazil: Comparison with India and analysis](#brazil)
4. [Second Wave in European Countries](#europe)

## A brief summary of Covid around the world. 

<div id ='summary'>
Before we begin, let's first view a world map with total confirmed cases in each country.</div>

In [ ]:
fig = go.Figure(data=go.Choropleth(locations = covid_cum['country'],
                            locationmode='country names',
                            z = covid_cum['total_confirmed'],
                            text = covid_cum['country'],
#                             color_continuous_scale='reds',
                            colorscale = 'RdBu',
                            autocolorscale=False,
#                             reversescale=True,
                            marker_line_color='darkgray',
                            marker_line_width=0.5,
                        
                          ))

fig.update_layout(
    title_text='Total confirmed cases around the world',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ))
fig.show()

The worst hit countries by total confirmed cases would be:

In [ ]:
covid_cum_top_10 = covid_cum[['country','total_confirmed']].sort_values('total_confirmed')[-10:]
fig = go.Figure(go.Bar(
            x=covid_cum_top_10['total_confirmed'],
            y=covid_cum_top_10['country'],
    marker_color = ['#00296f']* 10,
    text = covid_cum_top_10['total_confirmed'],
            orientation='h')
               ,layout=go.Layout(height=500, width=700))
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text='Top 10 countries by total confirmed case',xaxis_title='total confirmed cases')
fig.show()

* USA leads by cases over 30 million. It is also worth noting that the total number of cases in America are atleast twice of the succeding countries. 
* India and Brazil follow with 15 million cases reported which are again twice the number of cases of the countries following them.

This isn't a very accurate stat for the countries worst hit by Covid, since it does not take the population into account. Let's look at the top countries by `total cases per 1 million population`.

In [ ]:
color_dict={'Europe':'#7090ae','North America':'#a5b1bc','Asia':'#b1c5e7','South America':'#d9dee7','Africa':'#9ebbe7','Australia/Oceania':'#979eea'}

covid_cum_top_20_normalised = covid_cum[['country','total_cases_per_1m_population','continent']].sort_values('total_cases_per_1m_population')[-20:]
color_list = [color_dict[x] for x in covid_cum_top_20_normalised['continent'].values]
fig = go.Figure(go.Bar(
            x=covid_cum_top_20_normalised['total_cases_per_1m_population'],
            y=covid_cum_top_20_normalised['country'],
             marker_color = color_list,
             text=covid_cum_top_20_normalised['total_cases_per_1m_population'],
            orientation='h')
               ,layout=go.Layout(height=500, width=700))
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text='Top 20 countries by total cases per 1 million population',xaxis_title='total cases per 1m population')
fig.show()

* Astonishing, most of the countries that made it into top 10 by the total number of cases, do not appear here when the count is normalized by population. USA and France are the only ones repeating
* 14 out of the top 20 are European countries, 4 are in North America and remaining 2 in Asia. 
* Among the Scandinavian countries, Sweden and Netherlands appear in the top 20.
* Also, 1 out of every 10 person in America has been tested positive.

In [ ]:
covid_cum_top_20_deaths = covid_cum[['country','total_deaths_per_1m_population','continent']].dropna().sort_values('total_deaths_per_1m_population')[-20:]
color_list = [color_dict[x] for x in covid_cum_top_20_deaths['continent'].values]
fig = go.Figure(go.Bar(
            x=covid_cum_top_20_deaths['total_deaths_per_1m_population'],
            y=covid_cum_top_20_deaths['country'],
            text=covid_cum_top_20_deaths['total_deaths_per_1m_population'],
             marker_color= color_list,
            orientation='h')
               ,layout=go.Layout(height=500, width=700))
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text='Top 20 countries by total deaths per 1 million population',xaxis_title='total deaths per 1m population')

fig.show()



* One would assume that the countries worst hit by Covid would also record the most number of deaths, but this is not the case. There are other factors like dominant age-group,percentage of population with chronic illness, healthcare measures taken by the government like the rate of testing, public awareness and cooperation.
* This plot has more European countries than the one above it, 16 out of 20.
* Some countries that were not so severe with cases per 1m population like Spain,UK, Italy, Poland and Brazil have managed to appear in the top 20 for the worst death rates.
* Others like Israel, Swedeen, Netherlands and France have managed to have a relatively lower death rate.

A major factor in curbing COVID was the rate of testing. Let's see which are the top 20 countries in terms of tests per 1 million population.

In [ ]:

covid_cum_top_20_tests = covid_cum[['country','total_tests_per_1m_population','continent']].dropna().sort_values('total_tests_per_1m_population')[-20:]
color_list = [color_dict[x] for x in covid_cum_top_20_tests['continent'].values]
fig = go.Figure(go.Bar(
            x=covid_cum_top_20_tests['total_tests_per_1m_population'],
            y=covid_cum_top_20_tests['country'],
            text=covid_cum_top_20_tests['total_tests_per_1m_population'],
             marker_color= color_list,
    orientation='h'),layout=go.Layout(height=500, width=700))
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text='Top 20 countries by total tests per 1 million population',xaxis_title='total tests per 1m population')

fig.show()

* Either something is wrong with the data or some countries were extremely privileged when it came to testing. 6 tests per person, that's an awful lot.
* Even this plot is dominated by European countries followed by Asian countries.
* Some countries worst hit in rates of cases and deaths, managed to conduct good amount of tests like UK, Gibraltar, Israel and Bahrain(both didn't appear in top 20 for deaths). 




## Covid through the last one year
It would be interesting to analyze the new cases counted every day for the last one year (approx) throughout the world.

In [ ]:
world_cum=pd.DataFrame(covid.groupby('date')['daily_new_cases'].sum())
world_cum = world_cum.reset_index()
fig = px.bar(world_cum, x='date', y='daily_new_cases',width=700, height=400)
fig.show()

If we take data from most countries around the world into consideration then:-
* Covid kept infecting more and more people until mid January 2021- sometimes the slope is steeper like October-January , at times it is also levelled like August-October.
* After January 2021, the curve finally started a downward shift as the daily new cases kept reducing.
* At February the curve started an upward transition again and most of the world(or countries that contribute majorly to the population  of the world) experienced what we'll call a **Second Wave**.

Let's do a more granular analysis of this trend. I have handpicked some countries that had most of the covid cases , let's visualize whether they did or did not see a second spike.

In [ ]:
country_daily_cum = pd.DataFrame(covid.groupby(['date_new','country'])['daily_new_cases'].sum())
country_daily_cum = country_daily_cum.reset_index()
top_5_countries = ['USA','India','Brazil','China','France']
only_top=country_daily_cum[country_daily_cum['country'].isin(top_5_countries)]
fig = px.line(only_top, x="date_new", y="daily_new_cases",color='country',width=800, height=400)
fig.show()

It's interesting to see how different countries have fared through time.
* The recent second wave in India is quite prominent as the spike is huge.
* The United States has a stable and relatively low number of daily cases since it's peak in January.
* Brazil and France also had a second wave, which isn't very clear from the graph. We'll analyze them seperately below.
* Quite unusually despite being the country of the origin of this virus,the new daily cases in China is extremely low. This [article by Wired](https://www.wired.co.uk/article/china-crushed-coronavirus-pandemic) explains the unusual trend. 

## Second wave in India 
<div id = 'india'>
We'll start the analysis of the second wave by India, since it's ongoing.</div>

In [ ]:
india = covid[covid['country']=='India']
fig = px.bar(india, x='date', y='daily_new_cases',title='Daily new cases from Apr 2020 to Apr 2021',width=700, height=400)
fig.show()

* India recieved it's peak(of the first wave) in daily cases in mid September 2020, when the count touched hundred thousand new cases in a day.
* After this the sloped started a downhill ride almost as steep as the increase from April 2020-September 2020.
* Around mid February 2021, the curve started a gradual then later a steep increase in daily cases.
* On April 24 2021, the daily cases are ~349k, which is almost 3.5 times the cases during the first peak in Mid September.
* After the first week of April, most states in India are under a complete lockdown.

We'll compare this with other counts such as active cases, daily new deaths and daily vaccinations.

In [ ]:
india = covid[covid['country']=='India']
vaccine_ind=vaccine[vaccine['country']=='India']
fig = make_subplots(rows=3, cols=1,subplot_titles=('Active Cases','Daily New Deaths','Daily vaccinations'))
# fig.add_trace(go.Bar(x=india['date'].values, y=india['daily_new_cases'].values),
#               row=1, col=1)

fig.add_trace(go.Bar(x=india['date'].values, y=india['active_cases'].values,showlegend=False),
              row=1, col=1)

fig.add_trace(go.Bar(x=india['date'].values, y=india['daily_new_deaths'].values,showlegend=False),
              row=2, col=1)

fig.add_trace(go.Scatter(x=vaccine_ind['date'].values, y=vaccine_ind['daily_vaccinations'].values, showlegend= False,fill='tozeroy',
                    mode='none'),
              row=3, col=1)
fig.update_layout(height=800, width=700)
fig.show()


* The curve for active cases mostly follows similar trends as the plot above it, except it's much more smoother.
* The count for active cases on April 18 is 1.93 million, almost 2 times the active cases during the peak in mid September.
* The curve for daily new deaths is more spiky and irregular.
* The number of daily deaths reported on June 16 2020 are 2006 and it is way above the usual limit. It could easily be a typo.
* It is worth noting that on April 18th the daily death toll is 1625, which is not more than 1.5 times the death toll during the peak of the first wave.
* The amount of vaccines being administered daily is also increasing rapidly, with periodic dips. Until now 107 million people have recieved their first shot and 16 million people are fully vaccinated. These are people above the age of 45 as currently they are the only ones eligible.

**So, what caused this catastrophic Second wave in India?**
 
> During late January and February when the daily cases in India reduced a lot, the people and even government officials made the mistake of assuming the worst was over. Despite seeing a second wave in European Countries, the possibility was ignored. Regulations on social-distancing and wearing masks were relaxed. People started believing that the population would develop herd immunity. Also, vaccines were considered sufficient in the battle against covid.

## Second wave in Brazil 
<div id = 'brazil'> 
Brazil is the fifth most populous country in the world (India is second) and is amongst the countries worst hit by Covid. </div>

In [ ]:
brazil = covid[covid['country']=='Brazil']
fig = px.bar(brazil, x='date', y='daily_new_cases',width=700, height=400,title='Brazil - Daily new cases through the past year')
fig.show()

* The daily new cases curve isn't as regular as India's along with a lot of spikes. There are several peaks too. 
* Brazil definately has a second wave as [reported](https://www.newindianexpress.com/world/2021/mar/11/fatal-second-wave-brazils-daily-covid-19-death-toll-tops-2000-for-first-time-2275162.html) by different news agencies too.
* Around mid-August the daily cases started to decline and public spaces were open again, which again led to an increase and a second wave.

Let's view the daily deaths plot.

In [ ]:
vaccine_brz=vaccine[vaccine['country']=='Brazil']
fig = make_subplots(rows=3, cols=1,subplot_titles=('Active Cases','Daily New Deaths','Daily vaccinations'))

fig.add_trace(go.Bar(x=brazil['date'].values, y=brazil['active_cases'].values,showlegend=False),
              row=1, col=1)

fig.add_trace(go.Bar(x=brazil['date'].values, y=brazil['daily_new_deaths'].values,showlegend=False),
              row=2, col=1)

fig.add_trace(go.Scatter(x=vaccine_brz['date'].values, y=vaccine_ind['daily_vaccinations'].values,showlegend=False, fill='tozeroy',
                    mode='none'),
              row=3, col=1)
fig.update_layout(height=800, width=700)
fig.show()


* Active cases have mostly been on the rise in Brazil and follow the same trajectory as the new cases detected daily.The active cases on 18th april have crossed one million.
* Starting March, Brazil saw a huge spike in daily deaths, with 4211-the highest number of deaths recorded on April 6th 2020.
* The daily vaccinations administered plateaued around a quarter of a million until mid March, after which it started increasing.

In the early analysis between top countries, we saw that Brazil appeared in top 20 countries with most deaths per million population, despite not appearing in the top 20 for infections. 

> According to [this](https://www.nytimes.com/article/brazil-coronavirus-cases.html) *New York Times* article - "Some experts thought Brazil would be well equipped to rise to the challenge of a pandemic, based on its [track record in past public health emergencies](https://www.nytimes.com/2020/05/16/world/americas/virus-brazil-deaths.html). It has a public health care system that, while underfunded, provides robust coverage across the country. And it had time to study the responses of the first countries hit by the virus." But the President's position was often at odds with measures to comabt the virus.  "The Ministry of Health has not presented a comprehensive plan to fight the virus. One of its main initiatives in the first months was boosting the production of hydroxychloroquine, despite a medical consensus that the drug is ineffective and could even be dangerous, and encouraging doctors in the public health care system to prescribe it."

Use of hydroxychloroquine is not new. A lot of countries including USA and India were using it initially.

Now, let's compare Brazil and India on deaths and vaccination.

In [ ]:
figure= go.Figure()
fig = go.Figure(go.Bar(
            x=[covid_cum[covid_cum['country']=='India']['total_deaths_per_1m_population'].values[0],covid_cum[covid_cum['country']=='Brazil']['total_deaths_per_1m_population'].values[0]],
            y=['India','Brazil'],
            orientation='h'),layout=go.Layout(height=400, width=600))
fig.update_layout(title_text='India vs Brazil: Mortality Comparison',xaxis_title='total deaths per 1m population')

           

fig.show()

* The ratio for deaths per 1m population for India and Brazil is 1:13
* Brazil clearly has much higher mortality rate than India.

Could this be due to difference in vaccination?


In [ ]:
fig = make_subplots(rows=2, cols=1,subplot_titles=('People Vaccinated per hundred','People fully vaccinated per hundred'))
fig.add_trace(go.Scatter(x=vaccine_ind['date'].values, y=vaccine_ind['people_vaccinated_per_hundred'].values, fill='tozeroy',
                    mode='none',name='India' # 
                    ),1,1)
fig.add_trace(go.Scatter(x=vaccine_brz['date'].values, y=vaccine_brz['people_vaccinated_per_hundred'].values, fill='tozeroy',
                    mode= 'none',name = 'Brazil'),1,1)
fig.add_trace(go.Scatter(x=vaccine_ind['date'].values, y=vaccine_ind['people_fully_vaccinated_per_hundred'].values, fill='tozeroy',
                    mode='none',name='India' # 
                    ),2,1)
fig.add_trace(go.Scatter(x=vaccine_brz['date'].values, y=vaccine_brz['people_fully_vaccinated_per_hundred'].values, fill='tozeroy',
                    mode= 'none',name = 'Brazil'),2,1)
fig.update_layout(height=500, width=700)

fig.show()

* Vaccination covergae in Brazil has clearly been way better than India's. 
* Vaccination started a little earlier in Brazil than it did in India and has been more rapidly increasing ever since.
* For a group of hundred people, roughly twice as many people have gotten their first shot in Brazil.
* For the second shot, the ratio is even bigger , 1:4.

According to the same article that I've mentioned above
> Brazil has emerged as a potentially vital player in the global effort to find a vaccine. Four of the most promising and advanced vaccine studies in the world are relying on scientists and volunteers in Brazil, according to the World Health Organization’s report on the progress of vaccine research.

> The country’s role in research helped it cut two deals to get preferential access to possible vaccines. One such agreement is between the state of São Paulo and China’s Sinovac, for 46 million doses, and another is between the federal government and AstraZeneca, for 100 million doses. Both vaccines would be produced by Brazilian manufacturers.

>In late September, Brazil also decided to invest in the Covax Facility, a worldwide effort from governments and manufacturers to approve and distribute a vaccine.

This could possibly expalin the difference we saw in the data.

<div id ='europe'>
Earlier, we saw that France might be having a second wave. But it's not just France,most of the Eurpoean countries saw a second wave</div>  

## Second Wave in European Countries 
  

In [ ]:
france = covid[covid['country']=='France']
germany = covid[covid['country']=='Germany']
portugal = covid[covid['country']=='Portugal']
turkey = covid[covid['country']=='Turkey']
uk = covid[covid['country']=='UK']
fig = make_subplots(rows=5, cols=1,subplot_titles=('France','Germany','UK','Portugal','Turkey'))

fig.add_trace(go.Bar(x=france['date'].values, y=france['daily_new_cases'].values,name='France'),
              row=1, col=1)

fig.add_trace(go.Bar(x=germany['date'].values, y=germany['daily_new_cases'].values,name='Germany'),
              row=2, col=1)

fig.add_trace(go.Bar(x=uk['date'].values, y=uk['daily_new_cases'].values,name='UK'), row=3, col=1)

fig.add_trace(go.Bar(x=portugal['date'].values, y=portugal['daily_new_cases'].values,name='Portugal'),
              row=4, col=1)

fig.add_trace(go.Bar(x=turkey['date'].values, y=turkey['daily_new_cases'].values,name='Turkey'),
              row=5, col=1)

             
fig.update_layout(height=1200, width=700)
fig.show()

There are a lot more European countries that saw a second wave, I've chosen a subset of them here.
* The second wave in these countries started much early around October-November unlike India and Brazil.
* According to this [article](https://www.forbes.com/sites/joshuacohen/2020/10/19/second-wave-of-coronavirus-hits-europe-major-differences-in-impact-across-countries/?sh=1043b6051542) in Forbes, *By mid to late May, Europe had tamed the first wave of the coronavirus, but not eradicated it. Had Europe opted to pursue elimination of the virus, a second wave could have been avoided. But, on the whole, governments let down their guard. And then, complacency kicked in. Restrictions were substantially eased over the summer and into early autumn. Subsequently, as the weather cooled in September, people returned indoors to bars, restaurants, offices, and homes where the virus could easily spread.*
* For all of these countries the second wave caused way more infections than the first wave and for some countries like France and Poland, cases during the first wave are almost negligible in comparison.
* Turkey's second wave came in late November, a little later than in other countries. After a decline later in January, the cases started increasing again in March and are still on the rise. The daily new cases in April are twice it's peak during December.Turkey possibly has a third wave. 
* The same goes for Germany, after the second wave subsided in February , the daily new cases are reaching a new peak again.
* Portugal on the other hand is a country, that brutally got hit by the virus during the second wave. After the peak in late January , the country has managed to bring back it's daily cases to as few as 200-300.
* UK follows a curve similar to Portugal. Although, the number of cases are much higher due to the country's population. The second wave started around late September. It is also worth noting that the highest recorded daily cases(peak) in UK was on January 8 2021 with 67,928 cases while for France it was on November 7 2020 with 88,790 cases. (This could be the highest for Europe)

Daily new deaths through the first and second wave is also something worth looking at.


In [ ]:
# france = covid[covid['country']=='France']
# germany = covid[covid['country']=='Germany']
# portugal = covid[covid['country']=='Portugal']
# turkey = covid[covid['country']=='Turkey']
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Scatter(x=france['date'].values, y=france['daily_new_deaths'].values,name='France'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=germany['date'].values, y=germany['daily_new_deaths'].values,name='Germany'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=portugal['date'].values, y=portugal['daily_new_deaths'].values,name='Portugal'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=turkey['date'].values, y=turkey['daily_new_deaths'].values,name='Turkey'),
              row=1, col=1)
fig.add_trace(go.Scatter(x=uk['date'].values, y=uk['daily_new_deaths'].values,name='UK'),
              row=1, col=1)
fig.update_layout(height=500, width=800,title_text='Daily new deaths for European Countries')
fig.show()



* During the second wave, even though France's daily new cases were almost 10 times than during the first wave, France kept it's daily death count to almost half this time. The country was able to cope better.
* For Germany and UK, the case was opposite, as the number of daily cases increased so did the death count which surpasses the  average daily death count for the first wave. As we already saw, the infection cases in Germany are on the rise , but the death rate has gone down slowly. While for UK the curve for daily cases and daily deaths have both gone down a lot.
* Turkey on the other hand is not doing so well, along with the new cases rising every day, so is the death count.
* Portugal's death count after the second wave has almost reached zero.

**How are these countries dealing with the situation?**

Following 15th March 2021, Portugal has eased some of the lockdown measures. On the other hand, in France and Germany lockdowns are once again coming into force, as governments take action against the third wave. Turkey is also undergoing partial lockdowns. The multiple lockdowns in UK caused by the second wave have been starting to ease from late March 2021.


Let's compare deaths per million population for these countries.

In [ ]:
countries= ['France','Germany','Portugal','Turkey','UK']
europe_mortality=covid_cum[covid_cum['country'].isin(countries)].sort_values('total_deaths_per_1m_population')
fig = go.Figure(go.Bar(
            x=europe_mortality['total_deaths_per_1m_population'],
            y=europe_mortality['country'],
            orientation='h')
               ,layout=go.Layout(height=400, width=700))
fig.update_layout(title_text='Mortality rate- European countries',xaxis_title='total deaths per 1m population')


* Despite multiple lockdowns in UK, the country has the highest mortality rate followed by Portugal and France. 

> According to [this article](https://www.bbc.com/news/health-55757790), the high mortality rate of UK when compared with other European countries could be due to London being a global hub. *Genetic analysis has shown the virus was brought into the UK on at least 1,300 separate occasions, mainly from France, Spain and Italy, by the end of March*. Despite very low number of cases during the first wave(comparitively), the first wave saw a lot of deaths because of imposing a lockdown quite late. *Around 30% of deaths in the first wave happened in care homes; 40% if you include care home residents who died in hospital.* Besides factors like complacency on the public and government's part, which was present in most of the countries, UK had a relatively big share of people with chronic illness like diabetes, respiratory problems etc.
 
* While most of the deaths in Portugal came from the second wave, it was the opposite for France.
* Even though Turkey's mortality rate is comparitively low, if the same trend in daily deaths continue, we are going to see a rise in this too.

It is clear, that some countries did well during the second wave, some didn't.

Is vaccine a reason? *Let's check it out.*

In [ ]:
france = vaccine[vaccine['country']=='France']
germany = vaccine[vaccine['country']=='Germany']
portugal = vaccine[vaccine['country']=='Portugal']
turkey = vaccine[vaccine['country']=='Turkey']
uk = vaccine[vaccine['country']=='United Kingdom']
fig = make_subplots(rows=2, cols=1,subplot_titles=('People Vaccinated per hundred','People Fully vaccinated per hundred'))

fig.add_trace(go.Scatter(x=france['date'].values, y=france['people_vaccinated_per_hundred'].values,name='France'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=germany['date'].values, y=germany['people_vaccinated_per_hundred'].values,name='Germany'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=portugal['date'].values, y=portugal['people_vaccinated_per_hundred'].values,name='Portugal'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=turkey['date'].values, y=turkey['people_vaccinated_per_hundred'].values,name='Turkey'),
              row=1, col=1)
fig.add_trace(go.Scatter(x=uk['date'].values, y=uk['people_vaccinated_per_hundred'].values,name='UK'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=france['date'].values, y=france['people_fully_vaccinated_per_hundred'].values,name='France'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=germany['date'].values, y=germany['people_fully_vaccinated_per_hundred'].values,name='Germany'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=portugal['date'].values, y=portugal['people_fully_vaccinated_per_hundred'].values,name='Portugal'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=turkey['date'].values, y=turkey['people_fully_vaccinated_per_hundred'].values,name='Turkey'),
              row=2, col=1)
fig.add_trace(go.Scatter(x=uk['date'].values, y=uk['people_fully_vaccinated_per_hundred'].values,name='UK'),
              row=2, col=1)
fig.update_layout(height=800, width=700)
fig.show()

* UK has a remarkably high vaccination rate. They have ordered more than 400 million doses of seven of the most promising vaccines. Three have so far been approved for use:Oxford-AstraZeneca; Pfizer-BioNTech; and Moderna. Oxford-AstraZenca, one of the most widely used vaccine is made in different locations in Britain. You should check out [this article](https://www.bbc.com/news/health-55274833#:~:text=The%20UK%20is%20now%20receiving,number%20of%20sites%20in%20Britain.) for detailed analysis on vaccination in UK.
* Mostly UK has been giving first doses, around the last week of March it shifted focus from that to giving second doses.
* The vaccination curve for France, Germany and Portugal is following almost the same trend. France is doing a little worse in giving second shots of the vaccine.
* Turkey's curve has either been taking over or dipping below the general trend for the the remaining three countries.
* Turkey started vaccinating quite late in mid February. Initially, Turkey's rate of giving first vaccine shots was better than the three countries.
* Then in mid March, it shifted focus to administering second shots of the vaccine.